# Parametros de entrada

Los siguientes parametros deben ser modificados según como tenga el usuario almacenados los pesos de su modelo así como la carpeta donde se encuetran los nuevos datos:

In [ ]:
# Nombre del nuevo dataset
dataset = 'Nuevo-Dataset'

# Carpeta donde se encuentran los datos
source_folder = '/content/drive/MyDrive/DELT/Nuevo-Dataset'

# Carpeta donde se encuentran las imágenes antiguas
old_dataset_path = '/content/drive/MyDrive/DELT/Antiguo-Dataset'

# Carpeta donde queremos guardar los resultados
project_folder = '/content/drive/MyDrive/DELT/weights'

# Modelo del que partimos y carpeta donde lo tenemos guardado
model_v = 'Modelo-Anterior'
YOLO_model_w = '/content/drive/MyDrive/DELT/weights/'+model_v+'/weights/best.pt'


En el notebook se usan dos ficheros con la configuración del modelo (DELTv11.yaml) y la API key de Comet (.comet.config). Estos ficheros los guardamos en la siguiente ruta:

In [ ]:
config_folder = '/content/drive/MyDrive/DELT'

# Setup

In [ ]:
# Se adjunta esta libreria porque ultralytics puede dar errores
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Importamos librerias
import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [ ]:
# Instalamos la librería de ultralytics que contiene YOLOv11
!pip install ultralytics

In [ ]:
# Creamos directorio de datasets y importamos ficheros de configuracion
!mkdir datasets

!cp {config_folder}/DELTv11.yaml .
!cp {config_folder}/DELTv11_validation.yaml .
!cp {config_folder}/.comet.config .

In [ ]:
from ultralytics import YOLO

In [ ]:
# Esta función nos dará la lista de ficheros a mover desde Drive
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 1):

    full_name_list = []

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]

    return final_list

In [ ]:
# Movemos la lista de ficheros de Drive a Colab
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [ ]:
destination_images = '/content/datasets/DELT/images'
destination_labels = '/content/datasets/DELT/labels'


file_list = full_name_func(source_folder,dataset)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)


In [ ]:
# Creamos la estructura de train/val/test
if not os.path.exists('DELT'):
      for split in ['train','val', 'test']:
          os.makedirs(f'datasets/DELT/{split}/images')
          os.makedirs(f'datasets/DELT/{split}/labels')

In [ ]:
# Spliteamos las imagenes y labels
def get_filenames(folder):
    filenames = set()

    for path in glob.glob(os.path.join(folder, '*.*')):

        filename = os.path.split(path)[-1]
        filenames.add(filename)

    return filenames

def split_dataset(image_names, train_size, val_size):
    for i, image_name in enumerate(image_names):

        label_name = image_name.replace('.JPG', '.txt')
        label_name = label_name.replace('.jpg', '.txt')

        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'

        source_image_path = f'datasets/DELT/images/{image_name}'
        source_label_path = f'datasets/DELT/labels/{label_name}'

        target_image_folder = f'datasets/DELT/{split}/images'
        target_label_folder = f'datasets/DELT/{split}/labels'

        shutil.copy(source_image_path, target_image_folder)
        try:
          shutil.copy(source_label_path, target_label_folder) # No todas las imagenes tienen label
        except FileNotFoundError:
          pass

turtle_images = get_filenames('datasets/DELT/images')
turtle_images = np.array(list(turtle_images))

np.random.seed(42)
np.random.shuffle(turtle_images)

train_size = int(np.floor(len(turtle_images)*0.8))
val_size = int(np.floor(len(turtle_images)*0.2))

split_dataset(turtle_images, train_size, val_size)

In [ ]:
# Metemos en el set de validación los datos antiguos en la misma proporcion

file_list = full_name_func(old_dataset_path,dataset,frames = n_frames)

turtle_images = np.array(list(file_list))

np.random.seed(42)
np.random.shuffle(turtle_images)

# Movemos imagenes al dataset de validación
destination_images = '/content/datasets/DELT/val/images'
destination_labels = '/content/datasets/DELT/val/labels'
val_images = turtle_images[:val_size]
drive_to_colab(old_dataset_path,destination_images,destination_labels,dataset,val_images)

# Movemos imagenes al dataset de test
destination_images = '/content/datasets/DELT/test/images'
destination_labels = '/content/datasets/DELT/test/labels'
test_images = turtle_images[val_size:2*val_size]
drive_to_colab(old_dataset_path,destination_images,destination_labels,dataset,test_images)

# Entrenamiento

In [ ]:
# Usamos Comet ML
import comet_ml

comet_ml.init()

In [ ]:
# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset+ '-' + date_string

results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

# Test

In [ ]:
# Testeamos el nuevo modelo contra el dataset de test
weigths_path = project_folder + project_name + '/weights/best.pt'

# Usamos el .yaml de validacion
model = YOLO(weigths_path, data = 'DELTv11_validation.yaml')
metrics = model.val()

In [ ]:
# Testeamos el antiguo modelo contra el dataset de test
weigths_path = project_folder+model_v+'/weights/best.pt'

model = YOLO(weigths_path, data = 'DELTv11_validation.yaml')
metrics = model.val()